# Manual Splitting

We pre-define train, test, and validation based on "case id/ cover img id" for 5-fold validation.
1. Read images from the "BUS_Combined/imgs"
2. Group images base case id
3. Creating five folders for the 5-fold validation and save the splitted "case id" images in ".list" file. 
4. Adding images of each case and save it in a csv file.

In [1]:
import os
import numpy
from random import shuffle
import pandas as pd
from pathlib import Path
# Cover Image is a BUS Case, each cover image (Case) consist 3,4,5, or 6 bus images
PATH = './Datasets/BUS_Combined/imgs/'
SAVE_PATH = './Datasets/BUS_Combined/folds/'
Label_file ='./Datasets/BUS_Combined/BUS_Combined.csv'      # A sheet contain all 5 dataset names, covers, sub image (images of a case)


In [2]:
# Reading images
header_names = ["case id/cover img id", "img name","tumor type"]
url = pd.read_csv(Label_file, usecols = header_names)
print(len(url),url)

3641          img name case id/cover img id  tumor type
0            2879                 2879           0
1            3155                 3155           0
2            3156                 3155           0
3            3694                 3694           0
4            3695                 3694           0
...           ...                  ...         ...
3636  Case-270-U1             Case-270           0
3637  Case-272-U1             Case-272           0
3638  Case-273-U1             Case-273           1
3639  Case-274-U1             Case-274           1
3640  Case-276-U1             Case-276           1

[3641 rows x 3 columns]


In [3]:
# grouping images by cases
dfnew = pd.DataFrame(url)
dfnew = dfnew.groupby('case id/cover img id')['img name'].apply(list).to_dict()
print(len(dfnew),type(dfnew),len(dfnew.values()),len(dfnew))
#print(dfnew)
#all cases
df3 = pd.DataFrame(url)
print((df3.shape))
df3 = df3.groupby('tumor type')['img name'].apply(list).to_dict()
print((df3.keys()), type(df3), len(df3.values()), len(df3))


2102 <class 'dict'> 2102 2102
(3641, 3)
dict_keys([0, 1]) <class 'dict'> 2 2


In [4]:
def text_save(filename, data):      # filename: path to write CSV, data: data list to be written.
    file = open(filename, 'w+')

    for i in range(len(data)):
        s = str(data[i]).replace('[', '').replace(']', '')
        s = s.replace("'", '').replace(',', '') + '\n'
        file.write(s)
    file.close()
    print("Save {} successfully".format(filename.split('/')[-1]))

Creating five folders and each folder contains predefined cover lists with the names "folder#_test.list, folder#_train.list, and folder#_validation.list" files

In [5]:
def create_5_floder(folder, save_foler):
    coverList = []
    for key, value in folder.items():
        coverList.append(key)
    file_list = (coverList)
    #print(file_list)
    shuffle(file_list)
    print(len(file_list))
    total = len(file_list)
    print("total = ", total)
    testsize = int(total/5)
    reminder = total - testsize * 5

    trainsize = total - testsize
    validsize = int(trainsize * 0.15)
    print("Total = ", total, "Test size = ",testsize, reminder, "Train size = ",trainsize, "valid size = ",validsize)
    for i in range(5):
        if i != 0:
            pre_test_list = file_list[0:i*testsize]
            print("pre_test_list 4 = ", len(pre_test_list),i*testsize+reminder)
        else:
            pre_test_list = []
            print("pre_test_list0 = ", len(pre_test_list))
        
        if i != 4:
            test_list = file_list[i*testsize:(i+1)*testsize]
            print('test_list = ', len(test_list))
        else:
            test_list = file_list[i*testsize:]
            print('test_list = ', len(test_list))
        if i < 4:
            valid_list = file_list[(i+1)*testsize:(i+1)*testsize+validsize]
            train_list = file_list[(i+1)*testsize+validsize:] + pre_test_list
            print("train_list = ",  len(train_list),len(valid_list) )
        else:
            valid_list = file_list[:validsize]
            print("valid_list = ",  len(valid_list) )
            train_list = file_list[validsize:i*testsize]

        if not os.path.isdir(save_foler + '/folder'+str(i+1)):
            os.makedirs(save_foler + '/folder'+str(i+1))

        text_save(os.path.join(save_foler, 'folder'+str(i+1), 'folder'+str(i+1)+'_train.list'), train_list)
        text_save(os.path.join(save_foler, 'folder'+str(i+1), 'folder'+str(i+1)+'_validation.list'), valid_list)
        text_save(os.path.join(save_foler, 'folder'+str(i+1), 'folder'+str(i+1)+'_test.list'), test_list)


if __name__ == "__main__":
    create_5_floder(dfnew, SAVE_PATH)


2102
total =  2102
Total =  2102 Test size =  420 2 Train size =  1682 valid size =  252
pre_test_list0 =  0
test_list =  420
train_list =  1430 252
Save folder1\folder1_train.list successfully
Save folder1\folder1_validation.list successfully
Save folder1\folder1_test.list successfully
pre_test_list 4 =  420 422
test_list =  420
train_list =  1430 252
Save folder2\folder2_train.list successfully
Save folder2\folder2_validation.list successfully
Save folder2\folder2_test.list successfully
pre_test_list 4 =  840 842
test_list =  420
train_list =  1430 252
Save folder3\folder3_train.list successfully
Save folder3\folder3_validation.list successfully
Save folder3\folder3_test.list successfully
pre_test_list 4 =  1260 1262
test_list =  420
train_list =  1430 252
Save folder4\folder4_train.list successfully
Save folder4\folder4_validation.list successfully
Save folder4\folder4_test.list successfully
pre_test_list 4 =  1680 1682
test_list =  422
valid_list =  252
Save folder5\folder5_train.l

Helper functions to read, remove nested images and lookup images and cases. 


In [6]:
train = []
test = []
out = []
out2 = []


# filename: path to write CSV, data: data list to be written.
def read_text(filename):
    xret = []
    with open(filename, 'r') as reader:
        for line in reader.read().splitlines():
            xret.append(line)

    return(xret)

def reemovNestings(l):
    for i in l: 
        if type(i) == list: 
            reemovNestings(i) 
        else: 
            out.append(i) 
    return out
def reemovNestings2(l):
    for i in l: 
        if type(i) == list: 
            reemovNestings2(i) 
        else: 
            out2.append(i) 
    return out2

# function to return key for any value
def get_key(val, my_dict):
    for key, value in my_dict.items():
        #          print(len(value))
        if val[0] in value:
            #print(val,"??", value)
            return key
    return('Key Not Found')


Creating CSV files for Train, Valid, and Test lists

In [7]:
#mainpath = './Datasets_6/'

filenames = ['train', 'valid', 'test']
x = 0
output = []
output2 = []
dict1 = ['img name', 'tumor types']

for i in range(5):
    path1 = Path(os.path.join(SAVE_PATH, 'folder' +
                      str(i+1), 'folder'+str(i+1)+'_train.list'))
    train = read_text(path1)
    
    if path1.is_file():
        os.remove(path1)
        
    path2 = Path(os.path.join(SAVE_PATH, 'folder'+str(i+1),'folder'+str(i+1)+'_validation.list'))
    
    valid = read_text(path2)
    if path2.is_file():
        os.remove(path2)
    
    path3 = Path(os.path.join(SAVE_PATH, 'folder' + str(i+1), 'folder'+str(i+1)+'_test.list' ))      
    test = read_text(path3)
    if path3.is_file():
        os.remove(path3)
                           
    folderlist = [train, valid,  test]
    print('train cases = {}\ntest cases= {}\nvalid cases= {}\n '.format(len(train),len(test),len(valid)))
    x = 0
    for fold in folderlist:
        output.clear()
        output2.clear()
        out1 = []
        out2 = []
        classes = []
        valuesret = []
        x += 1
        for p in fold:
            for k, v in dfnew.items():   # k cover v sublist of cover

                kl = 0
                kret = 0
                if p == k:
                    dflist = list(v)
                    sizelist = len((dflist))

                    kret = get_key(dflist, df3)  # df3 = all images
                    kl = [kret] * sizelist
                
                    for il in range(len(kl)):
                        valuesret.append(dflist[il])
                        classes.append(kl[il])
                    break


        newlist = [valuesret, classes]

        df = pd.DataFrame(newlist).transpose()
        df.columns = dict1
        classes.clear()
        valuesret.clear()
        out1.clear()
        out2.clear()
        df.to_csv(SAVE_PATH + 'folder'+str(i+1) +
                  '/' + '%s.csv' % str(filenames[x-1]))
        row, col = df.shape
        print('folder'+str(i+1) +
                  '/' + '%s.csv' % str(filenames[x-1]), "=",row)
        
        del df


Successfully read data of  Datasets\BUS_Combined\folds\folder1\folder1_train.list
Successfully read data of  Datasets\BUS_Combined\folds\folder1\folder1_validation.list
Successfully read data of  Datasets\BUS_Combined\folds\folder1\folder1_test.list
train cases = 1430
test cases= 420
valid cases= 252
 
folder1/train.csv = 2512
folder1/valid.csv = 406
folder1/test.csv = 723
Successfully read data of  Datasets\BUS_Combined\folds\folder2\folder2_train.list
Successfully read data of  Datasets\BUS_Combined\folds\folder2\folder2_validation.list
Successfully read data of  Datasets\BUS_Combined\folds\folder2\folder2_test.list
train cases = 1430
test cases= 420
valid cases= 252
 
folder2/train.csv = 2601
folder2/valid.csv = 380
folder2/test.csv = 660
Successfully read data of  Datasets\BUS_Combined\folds\folder3\folder3_train.list
Successfully read data of  Datasets\BUS_Combined\folds\folder3\folder3_validation.list
Successfully read data of  Datasets\BUS_Combined\folds\folder3\folder3_test.lis